In [29]:
import pandas as pd
import json as js
from functools import reduce
import datetime as dt

In [30]:
# open json file
with open("Resources/kossal_copy_resources/all_electricity_supply.json") as supply_json:
    supply = js.load(supply_json)

In [31]:
# find number of columns from the data fram
num_of_col = len(supply)
num_of_col

12

In [32]:
s='Net generation : coal : United States : all sectors : monthly'
start='Net generation : '
end=' : United States : all sectors : monthly'
text=s[s.find(start)+len(start):s.rfind(end)]

In [33]:
# find columns names string and get the power sources
# supply[0]['name'] result in string 'Net generation : coal : United States : all sectors : monthly'
col_name = []
for i in range(len(supply)):
    s=supply[i]['name']    
    col_name.append(str(s[s.find(start)+len(start):s.rfind(end)]))
col_name

['coal',
 'natural gas',
 'nuclear',
 'conventional hydroelectric',
 'wind',
 'petroleum liquids',
 'all utility-scale solar',
 'geothermal',
 'wood and wood-derived fuels',
 'other gases',
 'other biomass',
 'other']

In [34]:
# create list of dataFrame Name
df_name=['df_'+str(i) for i in range(len(supply))]
df_name

['df_0',
 'df_1',
 'df_2',
 'df_3',
 'df_4',
 'df_5',
 'df_6',
 'df_7',
 'df_8',
 'df_9',
 'df_10',
 'df_11']

In [35]:
# Use for loop to create data frames
for index, col in enumerate(col_name):
       exec(f"{df_name[index]} = pd.DataFrame({supply[0]['data']}, columns=['date_id', '{col}'])")    

In [36]:
# create a list of DataFrames for join loop
df_list = [eval(temp) for temp in df_name]

In [37]:
# join DataFrames
df_energy_source = reduce(lambda x, y: pd.merge(x,y, on = 'date_id'), df_list)
df_energy_source.head()

,date_id,coal,natural gas,nuclear,conventional hydroelectric,wind,petroleum liquids,all utility-scale solar,geothermal,wood and wood-derived fuels,other gases,other biomass,other
0,202009,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702
1,202008,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383
2,202007,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355
3,202006,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394
4,202005,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478


In [38]:
# rename column to for SQL Tables
df_energy_source=df_energy_source.rename(columns={'natural gas':'natural_gas', 
                                 'conventional hydroelectric':'hydroelectric',
                                 'petroleum liquids':'petroleum_liquids',
                                 'all utility-scale solar': 'solar',
                                 'wood and wood-derived fuels':'wood',
                                 'other gases':'other_gases',
                                 'other biomass': 'other_biomass'
                                })
df_energy_source.head()

,date_id,coal,natural_gas,nuclear,hydroelectric,wind,petroleum_liquids,solar,geothermal,wood,other_gases,other_biomass,other
0,202009,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702
1,202008,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383
2,202007,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355
3,202006,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394
4,202005,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478


In [39]:
# columns of interest
col_int = ['date_id', 'natural_gas', 'nuclear', 'hydroelectric', 'wind', 'solar', 'wood', 'other_biomass', 'other']

In [40]:
# select column of interest
df_ener_sour = df_energy_source[col_int]
df_ener_sour.head()

,date_id,natural_gas,nuclear,hydroelectric,wind,solar,wood,other_biomass,other
0,202009,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702,68395.67702
1,202008,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383,91164.99383
2,202007,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355,89845.33355
3,202006,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394,65274.37394
4,202005,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478,46529.48478
